In [1]:
from pynq import Overlay
from pynq import MMIO

overlay = Overlay("design_1_wrapper.bit", download = True)

# Base address can be found in the Vivado Address Editor.
base_address = 0x40000000

# Memory size is determined by the BRAM IP's setting.
mem_size = 1024

mmio = MMIO(base_address, mem_size)

In [4]:
# Set the number of LED.
led_num = 9

# 0x0 is the memory address which store the number of LED.
# When the data is non-zero, the IP will start to execute and output LED's signal.
address_offset = 0x0
mmio.write(address_offset, led_num)

In [3]:
# Color format: 0xGGRRBB
color = 0xff0000  # Pure green
# Memory address that controls the first addressable RGB LED
address_offset = 0x4
mmio.write(address_offset, color)

color = 0x00ff00  # Pure red
# Memory address that controls the second addressable RGB LED
address_offset = 0x8
mmio.write(address_offset, color)

In [16]:
# Pack Operations into one class for easy use.
from pynq import Overlay
from pynq import MMIO

overlay = Overlay("design_1_wrapper.bit", download = True)

# Base address can be found in the Vivado Address Editor.
base_address = 0x40000000
# Memory size is determined by the BRAM IP's setting.
mem_size = 1024

class NeoPixel:
    def __init__(self, base_addr = 0x40000000, mem_size = 1024, num_leds = 1):
        self.mmio = MMIO(base_addr, mem_size)
        self.num_leds = num_leds
        self.base_addr = base_addr
        self.addr_step = 0x4
        num_leds = max(min(num_leds, 1023), 0)
        self.mmio.write(0x0, self.num_leds)
        
    def set_rgb(self, index, r, g, b):
        #if 0 <= index < self.num_leds:
            #r = max(min(r, 255), 0)
            #g = max(min(g, 255), 0)
            #b = max(min(b, 255), 0)
        color = g << 16 | r << 8 | b
        mmio.write(0x4 + (0x4 * index), color)
            
    def set_rgb_all(self, r, g, b):
        for i in range(self.num_leds):
            self.set_rgb(i, r, g, b)
            
    def close(self):
        for i in range(self.num_leds):
            self.set_rgb(i, 0, 0, 0)

In [14]:
led = NeoPixel(num_leds = 209)
led.set_rgb(0, 0xFF >> 2, 0x00 >> 2, 0x00 >> 2)
led.set_rgb(1, 0xFF >> 2, 0x7F >> 2, 0x00 >> 2)
led.set_rgb(2, 0xFF >> 2, 0xFF >> 2, 0x00 >> 2)
led.set_rgb(3, 0x00 >> 2, 0xFF >> 2, 0x00 >> 2)
led.set_rgb(4, 0x00 >> 2, 0xFF >> 2, 0xFF >> 2)
led.set_rgb(5, 0x00 >> 2, 0x00 >> 2, 0xFF >> 2)
led.set_rgb(6, 0x8B >> 2, 0x00 >> 2, 0xFF >> 2)
led.set_rgb(7, 0xFF >> 2, 0x00 >> 2, 0xFF >> 2)
led.set_rgb(8, 0xFF >> 2, 0xFF >> 2, 0xFF >> 2)

In [17]:
%%timeit
led.set_rgb(0, 0xFF >> 2, 0x00 >> 2, 0x00 >> 2)

24.5 µs ± 302 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
led.close()